In [1]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
SPAM_PATH = os.path.join("datasets", "spam")

def fetch_spam_data(ham_url=HAM_URL, spam_url=SPAM_URL, spam_path=SPAM_PATH):
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)
    for filename, url in (("ham.tar.bz2", ham_url), ("spam.tar.bz2", spam_url)):
        path = os.path.join(spam_path, filename)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=spam_path)
        tar_bz2_file.close()

In [2]:

fetch_spam_data()

In [3]:
HAM_DIR = os.path.join(SPAM_PATH, "easy_ham")
SPAM_DIR = os.path.join(SPAM_PATH, "spam")
ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]

In [4]:
len(ham_filenames)

2500

In [5]:
len(spam_filenames)

500

In [6]:
ham_filenames[0]

'00001.7c53336b37003a9286aba55d2945844c'

In [8]:
import email
import email.policy

In [26]:
file = open('datasets/spam/easy_ham/' + ham_filenames[0], 'rb') #open file

In [27]:
mail = email.parser.BytesParser(policy=email.policy.default).parse(file)
#https://docs.python.org/3/library/email.parser.html#email.parser.BytesParser


In [31]:
print(mail.get_content().strip())

Date:        Wed, 21 Aug 2002 10:54:46 -0500
    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>
    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>


  | I can't reproduce this error.

For me it is very repeatable... (like every time, without fail).

This is the debug log of the pick happening ...

18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}
18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury
18:19:04 Ftoc_PickMsgs {{1 hit}}
18:19:04 Marking 1 hits
18:19:04 tkerror: syntax error in expression "int ...

Note, if I run the pick command by hand ...

delta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury
1 hit

That's where the "1 hit" comes from (obviously).  The version of nmh I'm
using is ...

delta$ pick -version
pick -- nmh-1.0.4 [compiled on fuchsia.cs.mu.OZ.AU at Sun Mar 17 14:55:56 

In [32]:
def parse_email(spam, filename):
    if spam:
        dir = 'spam'
    else:
        dir = 'easy_ham'
    
    with open(os.path.join('datasets/spam/', dir, filename), 'rb') as f:
        return email.parser.BytesParser(policy = email.policy.default).parse(f)

In [37]:
test_email = parse_email(True, spam_filenames[1])

In [49]:
# print(test_email.get_content().strip())
spam[0]["Subject"]

'Life Insurance - Why Pay More?'

In [40]:
ham = [parse_email(False, name) for name in ham_filenames]
spam = [parse_email(True, name) for name in spam_filenames]

In [42]:
len(ham)

2500

In [43]:
len(spam)

500

X Add both together
y 0 for ham 1 for spam

In [44]:
import numpy as np

In [50]:
X = np.array(ham + spam, dtype=object)

In [52]:
y = np.array([0] * len(ham) + [1] * len(spam))

In [59]:
print(ham[0].get_content())

    Date:        Wed, 21 Aug 2002 10:54:46 -0500
    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>
    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>


  | I can't reproduce this error.

For me it is very repeatable... (like every time, without fail).

This is the debug log of the pick happening ...

18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}
18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury
18:19:04 Ftoc_PickMsgs {{1 hit}}
18:19:04 Marking 1 hits
18:19:04 tkerror: syntax error in expression "int ...

Note, if I run the pick command by hand ...

delta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury
1 hit

That's where the "1 hit" comes from (obviously).  The version of nmh I'm
using is ...

delta$ pick -version
pick -- nmh-1.0.4 [compiled on fuchsia.cs.mu.OZ.AU at Sun Mar 17 14:55

In [62]:
ham[2
    ].get_payload()

"Man Threatens Explosion In Moscow \n\nThursday August 22, 2002 1:40 PM\nMOSCOW (AP) - Security officers on Thursday seized an unidentified man who\nsaid he was armed with explosives and threatened to blow up his truck in\nfront of Russia's Federal Security Services headquarters in Moscow, NTV\ntelevision reported.\nThe officers seized an automatic rifle the man was carrying, then the man\ngot out of the truck and was taken into custody, NTV said. No other details\nwere immediately available.\nThe man had demanded talks with high government officials, the Interfax and\nITAR-Tass news agencies said. Ekho Moskvy radio reported that he wanted to\ntalk with Russian President Vladimir Putin.\nPolice and security forces rushed to the Security Service building, within\nblocks of the Kremlin, Red Square and the Bolshoi Ballet, and surrounded the\nman, who claimed to have one and a half tons of explosives, the news\nagencies said. Negotiations continued for about one and a half hours outside\nt

In [63]:
ham[1].items()

[('Return-Path', '<Steve_Burt@cursor-system.com>'),
 ('Delivered-To', 'zzzz@localhost.netnoteinc.com'),
 ('Received',
  'from localhost (localhost [127.0.0.1])\tby phobos.labs.netnoteinc.com (Postfix) with ESMTP id BE12E43C34\tfor <zzzz@localhost>; Thu, 22 Aug 2002 07:46:38 -0400 (EDT)'),
 ('Received',
  'from phobos [127.0.0.1]\tby localhost with IMAP (fetchmail-5.9.0)\tfor zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:46:38 +0100 (IST)'),
 ('Received',
  'from n20.grp.scd.yahoo.com (n20.grp.scd.yahoo.com    [66.218.66.76]) by dogma.slashnull.org (8.11.6/8.11.6) with SMTP id    g7MBkTZ05087 for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 12:46:29 +0100'),
 ('X-Egroups-Return',
  'sentto-2242572-52726-1030016790-zzzz=spamassassin.taint.org@returns.groups.yahoo.com'),
 ('Received',
  'from [66.218.67.196] by n20.grp.scd.yahoo.com with NNFMP;    22 Aug 2002 11:46:30 -0000'),
 ('X-Sender', 'steve.burt@cursor-system.com'),
 ('X-Apparently-To', 'zzzzteana@yahoogroups.com'),
 ('Recei

In [76]:
cnt = 0
for e in spam:
    if isinstance(e, str):
        pass
    p = e.get_payload()
    if isinstance(p, list):
        cnt +=1 
        print(p)
    else:
        # print(e.get_content_type())?
        pass

[<email.message.EmailMessage object at 0x7f87fc4414e0>, <email.message.EmailMessage object at 0x7f87fc443100>]
[<email.message.EmailMessage object at 0x7f87fc4421a0>, <email.message.EmailMessage object at 0x7f87fc441360>]
[<email.message.EmailMessage object at 0x7f87fc442350>, <email.message.EmailMessage object at 0x7f87fc441300>]
[<email.message.EmailMessage object at 0x7f87fc441660>, <email.message.EmailMessage object at 0x7f87fc440dc0>]
[<email.message.EmailMessage object at 0x7f87fc4415a0>, <email.message.EmailMessage object at 0x7f87fc440fa0>]
[<email.message.EmailMessage object at 0x7f87fc440490>, <email.message.EmailMessage object at 0x7f87fc440730>]
[<email.message.EmailMessage object at 0x7f87fc441e10>, <email.message.EmailMessage object at 0x7f87fc440970>]
[<email.message.EmailMessage object at 0x7f87fc4b75e0>, <email.message.EmailMessage object at 0x7f87fc46ee90>]
[<email.message.EmailMessage object at 0x7f87fc440f70>, <email.message.EmailMessage object at 0x7f87fc441030>]
[

In [73]:
cnt

98

In [ ]:
def structure(email):
    if isinstance(email, str):
        pass
    p = email.get_payload()
    if isinstance(p, list):
        print()